DNN

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
transforms = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(
    root = 'data',
    train = True,
    transform = transforms,
    download = True
)

trainloader = torch.utils.data.DataLoader(trainset,  batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(
    root = 'data',
    train = False,
    transform = transforms,
    download = True
)

testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [2]:
trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [3]:
testset

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [4]:
trainset.data.shape

torch.Size([60000, 28, 28])

In [5]:
testset.data.shape

torch.Size([10000, 28, 28])

In [6]:
trainset.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [7]:
trainloader

In [8]:
testloader

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class DNN(nn.Module):

    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(28**2, 100)
        #self.fc2 = nn.Linear(100, 100)
        #self.fc3 = nn.Linear(100, 100)
        #self.fc4 = nn.Linear(100, 100)
        #self.fc5 = nn.Linear(100, 100)
        self.out = nn.Linear(100, 10)

    def forward(self, x):
        x = x.view(-1, 28**2)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        #x = F.relu(self.fc5(x))
        x = self.out(x)

        return F.softmax(x)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = DNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()


def train_model(n_epochs=20):
    model.train()

    for epoch in range(n_epochs):
        print(f'Training Epoch: [{epoch+1}/{n_epochs}]')
        for batch_idx, (data, target) in enumerate(trainloader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
    
    print(f'\n---Training Completed---')

    train_loss = 0
    train_correct = 0

    with torch.no_grad():
        for data, target in trainloader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                train_loss += loss_fn(output, target).item()
                pred = output.argmax(dim=1, keepdim=True)
                train_correct += pred.eq(target.view_as(pred)).sum().item()
    
    train_loss /= len(trainloader.dataset)
    print(f'\nTrain Set: Average loss: {train_loss:.4f}, Accuracy: {train_correct}/{len(trainloader.dataset)} ({100. * train_correct / len(trainloader.dataset):.4f}%)\n')

def test():
    model.eval()

    test_loss = 0
    test_correct = 0

    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            test_correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(testloader.dataset)
    print(f'Test Set: Average loss: {test_loss:.4f}, Accuracy {test_correct}/{len(testloader.dataset)} ({100. * test_correct / len(testloader.dataset):.4f}%)\n')

In [11]:
train_model(n_epochs=20)
test()

Training Epoch: [1/20]


C:\Users\James Lee\AppData\Local\Temp\ipykernel_13240\4026878055.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Training Epoch: [2/20]
Training Epoch: [3/20]
Training Epoch: [4/20]
Training Epoch: [5/20]
Training Epoch: [6/20]
Training Epoch: [7/20]
Training Epoch: [8/20]
Training Epoch: [9/20]
Training Epoch: [10/20]
Training Epoch: [11/20]
Training Epoch: [12/20]
Training Epoch: [13/20]
Training Epoch: [14/20]
Training Epoch: [15/20]
Training Epoch: [16/20]
Training Epoch: [17/20]
Training Epoch: [18/20]
Training Epoch: [19/20]
Training Epoch: [20/20]

---Training Completed---

Train Set: Average loss: 0.0231, Accuracy: 59276/60000 (98.7933%)

Test Set: Average loss: 0.0234, Accuracy 9727/10000 (97.2700%)

